# Dataset Scraping

## Imports

In [1]:
from dwca.read import DwCAReader
from dwca.darwincore.utils import qualname as qn
import requests
import shutil
import os
import time
import math
import pandas
import json
import zipfile
from tqdm import tqdm
from glob import glob
import multiprocessing as mp


## Globals

In [4]:
timestr = time.strftime("%Y%m%d-%H%M%S")

PERCENT_TO_SCRAPE = 0.00015
NUMBER_TO_SKIP = 40000
DATASET_PATH = "/projectnb/sparkgrp/ml-herbarium-grp/ml-herbarium-data/"
DATASET_ARCHIVE = "data.zip"
DATASET_CSV = "data.csv"
OUTPUT_PATH = "/projectnb/sparkgrp/ml-herbarium-grp/ml-herbarium-data/scraped-data/" + "drago_testdata" + "/"
DATASET_URL = "https://occurrence-download.gbif.org/occurrence/download/request/0196625-210914110416597.zip"
DATASET_TYPE = ""
NUM_CORES = 50




## Download Dataset
#### Only run this if the dataset needs to be redownladed

In [209]:
if os.path.exists(DATASET_PATH + DATASET_ARCHIVE):
    os.remove(DATASET_PATH + DATASET_ARCHIVE)
ds = requests.get(DATASET_URL, stream=True)
total_size_in_bytes = int(ds.headers.get("content-length", 0))
block_size = 1024  # 1 Kibibyte
progress_bar = tqdm(total=total_size_in_bytes, unit="iB", unit_scale=True)
with open(DATASET_PATH + DATASET_ARCHIVE, "wb") as f:
    for data in ds.iter_content(block_size):
        progress_bar.update(len(data))
        f.write(data)
progress_bar.close()
if total_size_in_bytes != 0 and progress_bar.n != total_size_in_bytes:
    print("ERROR, something went wrong")


 10%|▉         | 113M/1.16G [00:20<03:10, 5.52MiB/s]
100%|██████████| 1.16G/1.16G [01:48<00:00, 10.7MiB/s]


## For DWCA files

### Open DWCA File

In [21]:
dwca = DwCAReader(DATASET_PATH + DATASET_ARCHIVE)
DATASET_TYPE = "dwca"


### Test DWCA
Will throw an error if the file is not opened correctly.

In [22]:
dwca.get_corerow_by_position(0)


### Save DWCA Rows to Pandas Dataframe

In [23]:
# df = dwca.pd_read('occurrence.txt')
df = dwca.pd_read("occurrence.txt", low_memory=False)


In [ ]:
def print_pandas_column_names(df):
    colnames = []
    for col in df.columns:
        colnames.append(col)
    print(colnames)

#### Close the archive to free resources


In [8]:
dwca.close()


In [11]:
df.describe()

,id,abstract,accrualMethod,accrualPeriodicity,accrualPolicy,alternative,audience,available,bibliographicCitation,conformsTo,...,acceptedTaxonKey,kingdomKey,phylumKey,classKey,orderKey,familyKey,genusKey,subgenusKey,speciesKey,relativeOrganismQuantity
count,8.409500e+04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,8.409500e+04,84095.0,84095.0,8.409500e+04,8.409500e+04,8.409400e+04,8.401600e+04,0.0,8.190900e+04,0.0
mean,2.214488e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.272101e+06,6.0,7707728.0,1.216576e+04,3.788900e+05,6.945661e+04,3.744964e+06,NaN,4.197946e+06,NaN
std,4.611772e+08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.850369e+06,0.0,0.0,2.936377e+05,1.607552e+06,5.277301e+05,1.844436e+06,NaN,1.809612e+06,NaN
min,1.989713e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.382000e+03,6.0,7707728.0,1.940000e+02,3.920000e+02,2.366000e+03,2.650108e+06,NaN,2.650247e+06,NaN
25%,1.998682e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.013672e+06,6.0,7707728.0,2.200000e+02,6.400000e+02,3.925000e+03,2.960439e+06,NaN,3.013580e+06,NaN
50%,1.999077e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.172326e+06,6.0,7707728.0,2.200000e+02,6.910000e+02,5.015000e+03,3.013395e+06,NaN,3.152047e+06,NaN
75%,2.012887e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.354470e+06,6.0,7707728.0,2.200000e+02,1.351000e+03,6.677000e+03,3.172323e+06,NaN,5.334240e+06,NaN
max,3.880670e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.140075e+07,6.0,7707728.0,7.228684e+06,7.561687e+06,1.041138e+07,1.140075e+07,NaN,1.137903e+07,NaN


In [12]:
df.shape

(84095, 258)

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84095 entries, 0 to 84094
Columns: 258 entries, id to geodeticDatum
dtypes: bool(2), float64(161), int64(8), object(87)
memory usage: 164.4+ MB


## For CSV files

### Extract CSV from zip file

In [9]:
# Run this only if you don't have csv file, and need to extract from data.zip that you just downloaded
if os.path.exists(DATASET_PATH + "*.csv"):
    os.remove(DATASET_PATH + "*.csv")
with zipfile.ZipFile(DATASET_PATH + DATASET_ARCHIVE, "r") as zip_ref:
    zip_ref.extractall(DATASET_PATH)
f = glob(os.path.join(DATASET_PATH, "*-*.csv"))[0]
os.rename(f, DATASET_PATH + DATASET_CSV)
print("CSV file extracted")


IndexError: list index out of range

### Save CSV Rows to Pandas Dataframe

In [14]:
df = pandas.read_csv(DATASET_PATH + DATASET_CSV, sep="\t")
DATASET_TYPE = "csv"


/scratch/ipykernel_66000/4273302753.py:1: DtypeWarning: Columns (2,14,16,17,19,21,24,25,26,32,33,34,36,37,38,39,40,41,43,45,46) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pandas.read_csv(DATASET_PATH + DATASET_CSV, sep="\t")


## Print Pandas Column Names

In [15]:
colnames = []
for col in df.columns:
    colnames.append(col)
print(colnames)


['gbifID', 'datasetKey', 'occurrenceID', 'kingdom', 'phylum', 'class', 'order', 'family', 'genus', 'species', 'infraspecificEpithet', 'taxonRank', 'scientificName', 'verbatimScientificName', 'verbatimScientificNameAuthorship', 'countryCode', 'locality', 'stateProvince', 'occurrenceStatus', 'individualCount', 'publishingOrgKey', 'decimalLatitude', 'decimalLongitude', 'coordinateUncertaintyInMeters', 'coordinatePrecision', 'elevation', 'elevationAccuracy', 'depth', 'depthAccuracy', 'eventDate', 'day', 'month', 'year', 'taxonKey', 'speciesKey', 'basisOfRecord', 'institutionCode', 'collectionCode', 'catalogNumber', 'recordNumber', 'identifiedBy', 'dateIdentified', 'license', 'rightsHolder', 'recordedBy', 'typeStatus', 'establishmentMeans', 'lastInterpreted', 'mediaType', 'issue']


## Get Images

### Export GBIF Ocurrence IDs

In [16]:
df.shape

(7982741, 50)

In [17]:
data = {}

NUMBER_TO_SKIP = math.floor(df.shape[0] / (df.shape[0] * PERCENT_TO_SCRAPE))
NUMBER_TO_SCRAPE = math.ceil(df.shape[0] / NUMBER_TO_SKIP)
print(str(NUMBER_TO_SCRAPE) + " IDs will be scraped.")
for i in range(1, df.shape[0], NUMBER_TO_SKIP):
    if DATASET_TYPE == "dwca":
        id = df.at[i, "id"]
    elif DATASET_TYPE == "csv":
        id = df.at[i, "gbifID"]
    data[i] = {"id": str(id)}
print("Successfully scraped " + str(len(data)) + " IDs.")


1198 IDs will be scraped.
Successfully scraped 1198 IDs.


### Fetch Image URLs and Specimen Data from GBIF API

In [20]:
rq = requests.get("https://api.gbif.org/v1/occurrence/" + str(data[key]["id"]))

NameError: name 'key' is not defined

In [19]:
print("Data will be fetched for", len(data), "occurrences.")


def scrape_occurrence(key):
    rq = requests.get("https://api.gbif.org/v1/occurrence/" + str(data[key]["id"]))
    return_dict = {}
    return_dict[key] = {}
    return_dict[key]["img_url"] = json.loads(rq.content)["media"][0]["identifier"]
    return_dict[key]["img_type"] = json.loads(rq.content)["media"][0]["format"]
    return_dict[key]["country"] = json.loads(rq.content)["country"]
    return_dict[key]["genus"] = json.loads(rq.content)["genus"]
    return_dict[key]["species"] = json.loads(rq.content)["species"]
    return return_dict

print("Starting multiprocessing...")
pool = mp.Pool(NUM_CORES)
print("Fetching data...")
for item in tqdm(pool.imap(scrape_occurrence, data), total=len(data)):
    data.update(item)
pool.close()
pool.join()

print("\nSuccessfully fetched data for", len(data), "occurrences.")


Data will be fetched for 1198 occurrences.
Starting multiprocessing...


  0%|          | 0/1198 [00:00<?, ?it/s]

Fetching data...


KeyError: 'id'

### Download Images

In [427]:
def download_images(key):
    img = requests.get(data[key]["img_url"], stream=True)
    with open(
        OUTPUT_PATH + str(key) + "." + data[1]["img_type"].split("/", 1)[1], "wb"
    ) as f:
        shutil.copyfileobj(img.raw, f)


if not os.path.exists(OUTPUT_PATH):
    os.makedirs(OUTPUT_PATH)

print("Starting multiprocessing...")
pool = mp.Pool(NUM_CORES)
print("Downloading images...")
for _ in tqdm(pool.imap(download_images, data), total=len(data)):
    pass
pool.close()


Starting multiprocessing...


100%|██████████| 1198/1198 [00:22<00:00, 54.02it/s]


### Export Geograpy Data

In [428]:
with open(OUTPUT_PATH + "countries.txt", "w") as f:
    for idx in data:
        f.write(data[idx]["country"] + "\n")
print("\nSuccessfully wrote countries to file.")



Successfully wrote countries to file.


### Export Taxon Data

In [429]:
with open(OUTPUT_PATH + "taxon.txt", "w") as f:
    for idx in data:
        f.write(data[idx]["genus"] + " " + data[idx]["species"] + "\n")
print("\nSuccessfully wrote taxon to file.")



Successfully wrote taxon to file.
